# Graph Convolutional Networks (GCN)

In this notebook, we’ll dive deeper into **Graph Convolutional Networks (GCN)** : one of the most popular architectures in Graph Neural Networks (GNNs). GCNs extend the concept of convolution to non-Euclidean data like graphs.

They were introduced in the paper: **“Semi-Supervised Classification with Graph Convolutional Networks” (Kipf & Welling, 2017)**.

## 🎯 Learning Objectives
- Understand the GCN layer operation and intuition.
- Implement a GCN from scratch using PyTorch Geometric.
- Train and evaluate GCN on the Cora citation dataset.
- Visualize the learned node embeddings.

## 🧠 1. Understanding GCN Layer Operation

The GCN layer updates each node’s representation by aggregating and transforming information from its neighbors.

The core operation is defined as:

\[
H^{(l+1)} = \sigma (\tilde{D}^{-1/2} \tilde{A} \tilde{D}^{-1/2} H^{(l)} W^{(l)})
\]

Where:
- \( \tilde{A} = A + I \): adjacency matrix with self-loops
- \( \tilde{D} \): degree matrix of \( \tilde{A} \)
- \( H^{(l)} \): node features at layer *l*
- \( W^{(l)} \): weight matrix
- \( \sigma \): non-linear activation (e.g., ReLU)

## ⚙️ 2. Import Dependencies

In [ ]:
import torch
import torch.nn.functional as F
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import GCNConv
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

print('Libraries imported successfully!')

## 📘 3. Load the Dataset (Cora)

The **Cora dataset** is widely used for node classification in citation networks.
- Nodes represent papers.
- Edges represent citation links.
- Each node has a feature vector and a class label.

In [ ]:
dataset = Planetoid(root='data/Cora', name='Cora')
data = dataset[0]

print('Number of nodes:', data.num_nodes)
print('Number of features:', data.num_features)
print('Number of edges:', data.num_edges)
print('Number of classes:', dataset.num_classes)

## 🏗️ 4. Define the GCN Model

In [ ]:
class GCN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super().__init__()
        self.conv1 = GCNConv(dataset.num_features, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, dataset.num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

model = GCN(hidden_channels=16)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
print(model)

## 🔁 5. Train the GCN Model

In [ ]:
def train():
    model.train()
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()
    return loss.item()

for epoch in range(201):
    loss = train()
    if epoch % 20 == 0:
        print(f'Epoch {epoch:03d}, Loss: {loss:.4f}')

## 📈 6. Evaluate the Model

In [ ]:
model.eval()
out = model(data)
pred = out.argmax(dim=1)
correct = (pred[data.test_mask] == data.y[data.test_mask]).sum()
acc = int(correct) / int(data.test_mask.sum())
print(f'Test Accuracy: {acc:.4f}')

## 🧭 7. Visualize Node Embeddings

In [ ]:
model.eval()
out = model(data)
z = out.detach().numpy()

tsne = TSNE(n_components=2, random_state=42)
z_2d = tsne.fit_transform(z)

plt.figure(figsize=(8,6))
plt.scatter(z_2d[:,0], z_2d[:,1], c=data.y, cmap='tab10')
plt.colorbar()
plt.title('t-SNE Visualization of Node Embeddings')
plt.show()

## 🧩 8. Key Insights
- GCNs generalize the concept of convolution to graphs.
- Each node updates its features based on its neighbors.
- GCNs can be applied to various tasks: node classification, link prediction, and graph classification.
- Adding more layers can lead to **over-smoothing**, where node representations become indistinguishable.

## 🔮 Next Steps
- Learn about **Graph Attention Networks (GAT)** for attention-based neighbor weighting.
- Explore **GraphSAGE** for inductive learning on unseen nodes.
- Apply GCNs on custom graph data (e.g., molecular, social).